In [2]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
import matplotlib.pyplot as plt 
import time
import numpy as np
import re
from itertools import combinations
import json

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [3]:
categories = ['atom-ph', 'cond-mat', 'hep-th', 'hep-ex', 'quant-ph']
# categories = ['atom-ph', 'quant-ph']
journals_dict = {'PRL': ['Physics Review Letters%',
                    'Phys. Rev. Lett.%',
                    'Phys.Rev.Lett.%',
                    'PRL%'],
                 'PR':  ['Physics Review%',
                         'Phys. Rev.%',
                         'Phys.Rev.%',
                         'PR%'],
                 'Nature': ['Nature%',
                            'Nat.%',
                            'Science%'],
                 'APL': ['APL%',
                         'Appl.Phys.Lett.%',
                         'Appl. Phys. Lett.%',
                         'Applied Physics Letters%'],
                 'AP': ['AP%',
                        'Appl.Phys.%',
                        'Appl. Phys.%',
                        'Applied Physics%'],
                 'PL': ['Physics Letters%',
                        'Phys. Lett.%',
                        'Phys.Lett.%'],
                 'All': ['%'],
                 }


In [12]:
def get_abstracts(category):
    query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%' + category + '%'),
                            or_(*[Article.journal_ref.like(x)
                                  for x in journals_dict['Nature']+journals_dict['PRL']]))
    return query

def get_abstracts2(category):
    query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%' + category + '%'),
                            or_(*[Article.journal_ref.like(x)
                                  for x in journals_dict['Nature']+journals_dict['PR']]))
    return query

In [10]:
# Get all categories
query = session.query(Category)
category_list = [(x.name, get_abstracts(x.name).count()) for x in query]
# for category in query:
#     print '%s: %i' % (category.name, get_abstracts(category.name).count())
for x in sorted(category_list, key=lambda x: -x[1]):
    print '%s: %i' % (x)

cond-mat: 18921
cond-mat.mes-hall: 4149
quant-ph: 4123
cond-mat.str-el: 3945
cond-mat.supr-con: 2321
supr-con: 2321
astro-ph: 2291
hep-ph: 2215
cond-mat.stat-mech: 2174
cond-mat.mtrl-sci: 2046
hep-ex: 1941
hep-th: 1406
cond-mat.quant-gas: 1215
cond-mat.soft: 1124
gr-qc: 924
nucl-th: 891
cond-mat.dis-nn: 881
cond-mat.other: 810
atom-ph: 801
physics.atom-ph: 800
physics.optics: 797
nucl-ex: 797
astro-ph.CO: 474
hep-lat: 369
q-bio: 334
astro-ph.HE: 307
math-ph: 283
math.MP: 283
physics.flu-dyn: 282
cs.FL: 282
physics.bio-ph: 258
nlin.CD: 244
chem-ph: 235
physics.chem-ph: 234
plasm-ph: 191
physics.plasm-ph: 190
cs.PL: 190
astro-ph.SR: 180
physics.soc-ph: 172
nlin.PS: 124
physics.comp-ph: 116
astro-ph.GA: 109
astro-ph.EP: 79
physics.data-an: 76
nlin.AO: 72
physics.atm-clus: 65
physics.ins-det: 62
q-bio.BM: 61
cs.SI: 61
q-bio.PE: 58
physics.acc-ph: 56
physics.class-ph: 50
cs.CL: 50
physics.geo-ph: 48
physics.space-ph: 47
nlin.SI: 42
q-bio.CB: 39
q-bio.SC: 37
astro-ph.IM: 37
q-bio.QM: 35
q-bi

In [13]:
# Get all categories
query = session.query(Category)
category_list = [(x.name, get_abstracts2(x.name).count()) for x in query]
# for category in query:
#     print '%s: %i' % (category.name, get_abstracts(category.name).count())
for x in sorted(category_list, key=lambda x: -x[1]):
    print '%s: %i' % (x)

cond-mat: 66018
hep-ph: 21547
quant-ph: 14844
cond-mat.str-el: 14124
cond-mat.mes-hall: 13647
hep-th: 13135
astro-ph: 13027
gr-qc: 10401
cond-mat.stat-mech: 8913
nucl-th: 8460
supr-con: 8030
cond-mat.supr-con: 8028
cond-mat.mtrl-sci: 6932
hep-ex: 6156
cond-mat.quant-gas: 4231
astro-ph.CO: 3949
cond-mat.soft: 3297
cond-mat.dis-nn: 3203
nucl-ex: 3153
hep-lat: 3004
cond-mat.other: 2693
atom-ph: 2404
physics.atom-ph: 2398
physics.optics: 2178
math-ph: 1802
math.MP: 1802
astro-ph.HE: 1603
q-bio: 1041
nlin.CD: 916
physics.soc-ph: 862
cs.FL: 791
physics.flu-dyn: 789
physics.bio-ph: 763
chem-ph: 763
physics.chem-ph: 753
astro-ph.SR: 653
physics.comp-ph: 614
nlin.PS: 526
cs.PL: 458
plasm-ph: 458
physics.plasm-ph: 457
physics.data-an: 397
astro-ph.IM: 362
astro-ph.GA: 357
cs.SI: 319
q-bio.PE: 280
physics.ins-det: 274
nlin.AO: 246
physics.acc-ph: 244
cs.CL: 233
physics.class-ph: 222
nlin.SI: 200
physics.atm-clus: 197
physics.gen-ph: 193
q-bio.BM: 174
math.PR: 167
astro-ph.EP: 149
physics.geo-ph: 